### Définition 1
Soient $\phi:V	\to V$ une transformation linéaire d'un espace vectoriel $V$ et $\lambda \in \mathbb{R}$ une valeur propre de $\phi.$ Alors l'espace propre de $\phi$ associé à $\lambda$ est le sous-ensemble de $V$ défini par $E_{\lambda}=\{v\in V: \phi(v)=\lambda v\}$.

De mani&egrave;re similaire, si $\lambda\in \mathbb{R}$ est une valeur propre de la matrice $A\in M_{n \times n}(\mathbb{R}),$ alors l'espace propre de $A$ associé à $\lambda$ est le sous-ensemble de $M_{n \times 1}(\mathbb{R})$ défini par $E_{\lambda}=\{X\in M_{n \times 1}(\mathbb{R}) : AX=\lambda X\}$.

### Proposition 2
Le sous-ensemble $E_\lambda$ est un sous-espace vectoriel.

In [ ]:
import numpy as np
import plotly
import plotly.graph_objects as go
import sympy as sp
import sys, os 
sys.path.append('../Librairie')
import AL_Fct as al
from IPython.utils import io
from IPython.display import display, Latex


In [ ]:
A = np.array([[1, 1, 3],[1, 1, 3], [1, 2, 1]])
al.printA(A)

eigs, eig_v = np.linalg.eig(A)

b = np.array([0, 0, 0])

with io.capture_output() as captured:
    A_ech = al.echelonMat('E', A)


al.printA(A_ech)



In [ ]:
A_sp = sp.Matrix([[1,2],[3,4]])

A_np = np.asarray(A_sp).astype(np.float64)


with io.capture_output() as captured:
    A_ech = al.echelonMat('E', A_np)

al.printA(A_ech)

In [ ]:
eig = A_sp.eigenvals()

list(eig.keys())

In [ ]:
def free_var(e_Mat, n):
    """Given an ech matrix of size n x n, return a list a boolean indicating the free variables"""
    var = [None]*n
    
    for i in range(n):
        if abs(e_Mat[i, i]) > 1e-14:
            var[i] = False
        else:
            var[i] = True
    return var


def texVector(v):
    """
    Return latex string for vertical vector
    Input: v, 1D np.array()
    """
    n = v.shape[0]
    return al.texMatrix(v.reshape(n, 1))


def check_basis(sol, prop):
    """
    Checks if two basis are equivalent
    sol: verfied basis of eigenspace 
    prop: proposed basis of eigenspace
    Both input are np.array with shape (dim, n) with dim the number of vector in the basis 
    and n the size of each vector in the basis
    return: boolean 
    """
    prop = np.array(prop, dtype=np.float64)
    
    # number of vector in basis 
    n = len(sol)

    # Check dimension of proposed eigenspace
    if n != len(prop):
        display(Latex("Le nomber de vecteur(s) propre(s) donné(s) est incorrecte. " + 
                      "La dimension de l'espace propre est égale au nombre de variable(s) libre(s)."))
        return False
    else:
        # Check if the sol vector can be written as linear combination of prop vector
        # Do least squares to solve overdetermined system and check if sol is exact
        A = np.transpose(prop)
        lin_comb_ok = np.zeros(n, dtype=bool)
        
        for i in range(n):
            print(A)
            print(sol[i])
            
            x, _, _, _ = np.linalg.lstsq(A, sol[i], rcond=None)
            print(x)
            res = np.sum((A @ x - sol[i])**2)
            lin_comb_ok[i] = res < 10**-13
        
        return np.all(lin_comb_ok)
    
    
def eigen_basis(A, l, prop_basis=None, disp=True):
    """
    Display step by step method for finding a basis of the eigenspace of A associated to eigenvalue l
    Eventually check if the proposed basis is correct.
    """
    n = A.shape[0]
    if n!=A.shape[1]:
        raise ValueError('A should be a square matrix')
    
    eig,v = np.linalg.eig(A)
    
    if np.all(abs(l-eig) > 1e-10):
        raise ValueError("lambda n'est pas une valeur propre de A.")
    
    I = np.eye(n)
    Mat = A-l*I
    b = np.zeros(n)

    if disp:
        display(Latex("On a $ A = " + al.texMatrix(A) + "$."))
        display(Latex("On cherche une base de l'espace propre associé à $\lambda = " + str(l) + "$."))

    with io.capture_output() as captured:
        e_Mat = al.echelonMat('ER', Mat, b)

    # Get free variables
    free = free_var(e_Mat, n)
    n_free = np.sum(free)
    free_idx = np.argwhere(free)[:, 0]
    basic_idx = np.argwhere(np.logical_not(free))[:, 0]


    # String to print free vars
    free_str = ""
    for i in range(n):
        if free[i]:
            free_str += "x_" + str(i+1) + " \ "

    if disp:
        display(Latex("On échelonne la matrice du système $A -\lambda I = 0 \Rightarrow " 
                      + al.texMatrix(Mat, np.reshape(b, (n,1))) + "$"))
        display(Latex("On obtient: $" + al.texMatrix(e_Mat[:, :n], e_Mat[: ,n:]) + "$"))
        display(Latex("Variable(s) libre(s): $" + free_str + "$"))

    # Matrix without extension of zeros (Red for reduced)
    e_Red = e_Mat[:, :n]

    # Build a list of n_free basis vector: 
    # first dim: which eigenvector (size of n_free) 
    # second dim: which element of the eigenvector (size of n)
    basis = np.zeros((n_free, n))

    for idx, j in enumerate(free_idx):
        for i in range(n):
            if i not in free_idx:
                basis[idx, i] = - e_Red[i, j]/e_Red[i, i]
            elif i==j:
                basis[idx][i] = 1
            else:
                basis[idx][i] = 0


    # Show calculated basis 
    basis_str = ""
    for idx, i in enumerate(free_idx):
        basis_str += "x_" + str(i+1) + " \cdot"  + texVector(basis[idx]) 
        if idx < n_free - 1:
            basis_str += " + "

    if disp:
        display(Latex("On peut donc exprimer la base de l'espace propre comme: $" + basis_str + "$"))

    if prop_basis is not None and disp:
        correct_answer = check_basis(basis, prop_basis)

        if correct_answer:
            display(Latex("La base donnée est correcte."))

        else:
            display(Latex("La base donnée est incorrecte."))


    return basis, basic_idx, free_idx
        

In [ ]:
A = np.array([[4, -1, 6],[2, 1, 6], [2, -1, 8]])

#eig = np.linalg.eig(A)


A_sp = sp.Matrix(A)
eig = A_sp.eigenvects()
print(eig)
eig_list = np.array(eig[1][2])

for sp_mat in eig_list:
    sp_mat = np.array(sp_mat)
print(eig_list)
A = np.eye(3)
    
basis, basic_idx, free_idx = eigen_basis(A, 1, eig_list, disp=False)

print(basic_idx, free_idx)

In [ ]:
def create_eig_set(A, tol=10**-13):
    w, _ = np.linalg.eig(A)
    w_imag = np.imag(w)
    if np.all(np.abs(w_imag) <10**-13):
        w = np.real(w)
    else:
        raise ValueError("Complex eigenvalues detected")
    w_set = [w[0]]
    
    for i in range(1,3):
        # If no very close element already in w_set, then add w[i] in w_set
        if np.all(np.abs(w_set - w[i])> tol):
            w_set.append(w[i])
    
    return w_set
            

def create_eig_set_sp(A):
    A_sp = sp.Matrix(A)
    eig_dic = A_sp.eigenvals()
    return np.array(list(eig_dic.keys()), dtype=np.float64)


def plot3x3_eigspace(A, xL=-10, xR=10, p=None):
    # To have integer numbers
    if p is None:
        p = xR-xL + 1
    
    A = np.array(A)
    n = A.shape[0]
    # Check 3 by 3
    if n!=3 or n!=A.shape[1]:
        raise ValueError("A should be 3 by 3")
    
    w = create_eig_set_sp(A)
    
    gr = 'rgb(102,255,102)'
    org = 'rgb(255,117,26)'
    # red = 'rgb(255,0,0)'
    blue = 'rgb(51, 214, 255)'
    colors = [blue, gr, org]
    s = np.linspace(xL, xR, p)
    t = np.linspace(xL, xR, p)
    tGrid, sGrid = np.meshgrid(s, t)
    data = []
    
    for i, l in enumerate(w):
        basis, basic_idx, free_idx = eigen_basis(A, l, disp=False)
        
        n_free = len(basis)
        if n_free != len(free_idx):
            raise ValueError("len(basis) and len(free_idx) should be equal.")
        
        gr = 'rgb(102,255,102)'
    
        colorscale = [[0.0, colors[i]],
                      [0.1, colors[i]],
                      [0.2, colors[i]],
                      [0.3, colors[i]],
                      [0.4, colors[i]],
                      [0.5, colors[i]],
                      [0.6, colors[i]],
                      [0.7, colors[i]],
                      [0.8, colors[i]],
                      [0.9, colors[i]],
                      [1.0, colors[i]]]
        
        X=[None]*3

        if n_free == 2:
            X[free_idx[0]] = tGrid
            X[free_idx[1]] = sGrid
            X[basic_idx[0]] = tGrid *basis[0][basic_idx[0]] + sGrid * basis[1][basic_idx[0]]
            
            
            plot_obj = go.Surface(x=X[0], y=X[1], z=X[2],
                                     showscale=False, showlegend=True, colorscale=colorscale, opacity=1,
                                     name="Lambda = " + str(l))

        elif n_free == 1:
            
            plot_obj = go.Scatter3d(x= t*basis[0][0], y=t*basis[0][1], z=t*basis[0][2],
                        line=dict(color='red', width=4),
                        mode='lines',
                        name='Lambda = ' + str(l))
        
        data.append(plot_obj)
        layout = go.Layout(
            showlegend=True,  # not there WHY???? --> LEGEND NOT YET IMPLEMENTED FOR SURFACE OBJECTS!!
            legend=dict(orientation="h"),
            autosize=True,
            width=800,
            height=800,
            scene=go.layout.Scene(
                xaxis=dict(
                    gridcolor='rgb(255, 255, 255)',
                    zerolinecolor='rgb(255, 255, 255)',
                    showbackground=True,
                    backgroundcolor='rgb(230, 230,230)',
                    range=[xL, xR]
                ),
                yaxis=dict(
                    gridcolor='rgb(255, 255, 255)',
                    zerolinecolor='rgb(255, 255, 255)',
                    showbackground=True,
                    backgroundcolor='rgb(230, 230,230)',
                    range=[xL, xR]
                ),
                zaxis=dict(
                    gridcolor='rgb(255, 255, 255)',
                    zerolinecolor='rgb(255, 255, 255)',
                    showbackground=True,
                    backgroundcolor='rgb(230, 230,230)', 
                    range=[xL, xR]
                ),
                aspectmode="cube",
            )
        )

            
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig)
    return
        
        
        
        
        
        

In [ ]:
A = np.array([[4, -1, 6],[2, 1, 6], [2, -1, 8]])

plot3x3_eigspace(A)
